# HydroBucket 模型测试

这个 notebook 展示了 HydroBucket 水文模型的基本使用方法，包括：
- 单节点模型的设置和运行
- 多节点模型的设置和运行
- 参数梯度计算示例

## 导入必要的包

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.precompile("HydroModels")

using CSV
using DataFrames
using ComponentArrays
using HydroModels

  Activating project at `e:\JlCode\HydroModels\docs\notebook`


LoadError: LoadError: UndefVarError: `Num` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing Symbolics in the current active module Main
Hint: a global variable of this name may be made accessible by importing ModelingToolkit in the current active module Main
in expression starting at e:\JlCode\HydroModels\docs\notebook\models\exphydro.jl:8

## 创建模型

In [ ]:
step_func(x) = (tanh(5.0 * x) + 1.0) * 0.5
# define variables and parameters
@variables temp lday pet prcp snowfall rainfall snowpack melt
@parameters Tmin Tmax Df Smax Qmax f

# define model components
snow_bucket = @hydrobucket :surface begin
    fluxes = begin
        @hydroflux begin
            snowfall ~ step_func(Tmin - temp) * prcp
            rainfall ~ step_func(temp - Tmin) * prcp
        end
        @hydroflux melt ~ step_func(temp - Tmax) * step_func(snowpack) * min(snowpack, Df * (temp - Tmax))
        @hydroflux pet ~ 29.8 * lday * 24 * 0.611 * exp((17.3 * temp) / (temp + 237.3)) / (temp + 273.2)
    end
    dfluxes = begin
        @stateflux snowpack ~ snowfall - melt
    end
end

## 模型参数设置

下面我们设置模型的基本参数：
- f: 渗透系数
- Smax: 最大存储容量
- Qmax: 最大流量
- Df: 扩散系数
- Tmax: 最高温度阈值
- Tmin: 最低温度阈值

In [ ]:
# 创建参数向量
params = ComponentVector(f=0.0167, Smax=1709.461, Qmax=18.469, Df=2.6745, Tmax=0.1757, Tmin=-2.0929)
init_states = ComponentVector(snowpack=100.0)
pas = ComponentVector(params=params, initstates=init_states)

# 读取输入数据
file_path = "../data/exphydro/01013500.csv"
data = CSV.File(file_path)
df = DataFrame(data)
ts = collect(1:10000)

# 准备单节点输入数据
input = (lday=df[ts, "dayl(day)"], temp=df[ts, "tmean(C)"], prcp=df[ts, "prcp(mm/day)"])
input_arr = reduce(hcat, collect(input[HydroModels.get_input_names(ele)])) |> permutedims

In [ ]:
# 创建单节点模型
single_ele = HydroBucket(name=:surface, fluxes=fluxes_1, dfluxes=dfluxes_1)

# 运行模型
result = single_ele(input_arr, pas)